## Sentiment generation

Generates new columns with the flair sentiment (vflair), flair confidence (cflair), and analogous for blob

In [ ]:
path = "../data/"

In [ ]:
modules = ["flair","spacy","spacytextblob"]

import sys
import os.path
from subprocess import check_call
import importlib
import os

def instala(modules):
  print("Instalando módulos")
  for m in modules:
      # para el import quitamos [...] y ==...
      p = m.find("[")
      mi = m if p==-1 else m[:p]
      p = mi.find("==")
      mi = mi if p==-1 else mi[:p]

      torch_loader = importlib.util.find_spec(mi)
      if torch_loader is not None:
          print(m," encontrado")
      else:
          print(m," No encontrado, instalando...",end="")  
          try:        
            r = check_call([sys.executable, "-m", "pip", "install", "--user", m])
            print("¡hecho!")
          except:
            print("¡Problema al instalar ",m,"! ¿seguro que el módulo existe?",sep="")
              
  print("¡Terminado!")

instala(modules) 

In [ ]:
#pipelines = ['en_core_web_lg']
pipelines = ['en_core_web_lg']
def instala_pipelines(pipelines):
  print("Instalando pipelines de Spacy")
  for p in pipelines:
      print("Instalando ",p)
      try:
        r = check_call([sys.executable, "-m", "spacy", "download", p])
        print("¡Hecho!")
      except:
        print("¡Problema al instalar ",p,"! ¿seguro que el pipeline existe?",sep="")

  print("¡Terminado!")

instala_pipelines(pipelines)


In [ ]:
from flair.models import TextClassifier
from flair.data import Sentence
classifier = TextClassifier.load('en-sentiment')

import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
nlp = spacy.load('en_core_web_lg')
nlp.add_pipe('spacytextblob')

def sentiment(s):

  # flair
  text = Sentence(s)

  classifier.predict(text)
  value = text.labels[0].to_dict()
  #print(s)
  if value["value"]=="NEGATIVE":
      vflair = -1
  elif value["value"]=="POSITIVE":
      vflair = +1
  else :
      vflair = 0 # no sé si esto pasa
  cflair = value["confidence"]

  # textblob  
  doc = nlp(s) # this returns data of the "sentiment" of the text
  vblob = doc._.blob.polarity
  cblob = doc._.blob.subjectivity
  return vflair,cflair, vblob,cblob

In [ ]:
import pandas as pd
import numpy as np

for a in ['16','20','24']:
    for p in ['dem','rep']:
        fichIn = "tweets"+a+"_"+p+".csv"
        fichOut = "tweets"+a+"_"+p+"_sft.csv"
        print(fichIn,end=" - ")
        df = pd.read_csv(path+fichIn)
        df[['vflair', 'cflair', 'vblob', 'cblob']] = df['Tweet'].apply(lambda tweet: pd.Series(sentiment(tweet)))
        df.to_csv(path+fichOut,index=False)


In [ ]:
fichOut

## Results analysis

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer,f1_score, accuracy_score, recall_score,cohen_kappa_score, confusion_matrix, ConfusionMatrixDisplay,classification_report
def show_results(y_real,y_pred,classes):
    print("Accuracy ",round(accuracy_score(y_real,y_pred),4),\
              "\nKappa ", round(cohen_kappa_score(y_real,y_pred),4))
    print('\nClassification Report: \n', classification_report(y_real, y_pred))
    cm = confusion_matrix(y_real, y_pred, labels=classes)
    print(cm)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                display_labels=classes)
    disp.plot()

    plt.show()
    
    print(classification_report(y_real, y_pred))
    plt.show()

In [ ]:

for a in ['16','20','24']:
    for p in ['dem','rep']:
        fichIn = "tweets"+a+"_"+p+"_sft.csv"
        df = pd.read_csv(path+fichIn)
        df2 = df[df.vblob!=0]
        df2["vblob_d"] = df2.vblob.map(lambda x: -1 if x<0 else +1)
        print(fichIn)
        show_results(df2.vflair,df2.vblob_d,[-1,1])

In [ ]:
df2.vflair.value_counts()

In [ ]:
df2.vblob_d.value_counts()

In [ ]:
pd.set_option('display.max_colwidth', None) 
filter = (df2.vflair ==-1) & (df2.vblob_d==1)
df2[filter].sample(20)

### Comparison with human classification

In [ ]:
fichHumanDem = "demManual.csv"
fichHumanRep = "trumpManual.csv"
rep = "tweets24_rep_sft.csv"
dem ="tweets24_dem_sft.csv"

dfHRep = pd.read_csv(path+fichHumanRep)
dfHDem = pd.read_csv(path+fichHumanDem)
dfRep = pd.read_csv(path+rep).drop_duplicates("Tweet")
dfDem = pd.read_csv(path+dem).drop_duplicates("Tweet")
print(len(dfHRep),len(dfHDem),len(dfRep),len(dfDem))

In [ ]:
dfHRep_sinNA = dfHRep[~dfHRep.Final.isna()]
dfHDem_sinNA = dfHDem[~dfHDem.Final.isna()]
dfRep_unique = df

comunRep = pd.merge(dfHRep_sinNA, dfRep, how="inner", left_on="Tweet", right_on="Tweet")
comunDem = pd.merge(dfHDem_sinNA, dfDem, how="inner", left_on="Tweet", right_on="Tweet")

In [ ]:
len(comunRep), len(comunDem)

In [ ]:
show_results(comunRep.Final,comunRep.vflair,[-1,1])

In [ ]:
comunRep["vblob_d"] = comunRep.vblob.map(lambda x: -1 if x<0 else +1 if x>0 else 0)
comunDem["vblob_d"] = comunDem.vblob.map(lambda x: -1 if x<0 else +1 if x>0 else 0)

In [ ]:
show_results(comunRep.Final,comunRep.vblob_d,[-1,0,1])

In [ ]:
show_results(comunDem.Final,comunDem.vflair,[-1,1])

In [ ]:
show_results(comunDem.Final,comunDem.vblob_d,[-1,0,1])